__The input file contains the following fields:__

| Name    | Description |
| -------- | ------- |
| UID  | Unique ID of the owning application    |
| DBType | Type of the Databse (MSSQL, MySQL, Oracle, DB2)     |
| Instance    | The instance of the Ddtabase |
| DBName | The name of the database |
| Schema | The schema where the table os located in the database |
| Table | The name of the table |
| Column | The name of a single column |
| ColumnType | The datatyp of the column |

| DB | UID |    DBType | Instance |    DBName |      Schema |      Table |           Column |         ColumnType | 
| ----- | ----- |    ----- | ----- |    ----- |      ----- |      ----- |           ----- |         ----- | 
| __MSSQL__ | UID |    DBType | Instance |    DBName |      Schema |      Table |           Column |         ColumnType | 
| __Oracle__ | uid |   dbtype |   |           dbname |      owner |       table_name |      column_name |     data_type |
| __MySQL__ | uid |    dbtype |    |          dbname |      owner |       table_name |      column_name |     data_type | 
| __FINAL COLUMN NAMES__ | UID | DB_TYPE | _droped_ | DB_NAME | SCHEMA |  TABLE_NAME | COLUMN_NAME | DATA-TYPE | 

In [52]:
!pip install pyLDAvis
!pip install wordninja
!pip install spacy
!pip install lingua-language-detector

/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=78949) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=78949) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=78949) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=78949) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


# Prepare the data

we have metadata about MSSQL, Oracle and MySQL Instances in CSV compressed in ZIP files - these CSVs must be merged into the same structure

In [53]:
# remove the existing data files
!rm -f data/*.csv  
!rm -f data/*.dic
!rm -f data/*.txt

# extract the german dictory used for word splitting
!unzip data/german.zip german.dic -d data

# unzip the data files avialable
!unzip -o data/mssql_allprod_collumns.zip -d data/
!unzip -o data/oracle_metadata.zip -d data/
!unzip -o data/mysql_metadata.zip -d data/

/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=78949) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()
/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=78949) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


zsh:1: no matches found: data/*.txt


/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=78949) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Archive:  data/german.zip
  inflating: data/german.dic         

/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=78949) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


...

/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=78949) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()



Archive:  data/oracle_metadata.zip
  inflating: data/oracle_metadata.csv  

/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=78949) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()



Archive:  data/mysql_metadata.zip
  inflating: data/mysql_metadata.csv  


/opt/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=78949) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [54]:
%%bash
# Prepare MSSQL data

INPUT_FILE="data/mssql_allprod_collumns.csv"
OUTPUT_FILE="data/mssql_allprod_collumns_no_third_column.csv.tmp"

head -3 $INPUT_FILE

# because the column separator is ',', we need to make sure e.g. `numeric(17,10)` is replaced with `numeric(17;10)` in the csv file
sed -E 's/\(([^)]*),([^)]*)\)/(\1;\2)/g' ${INPUT_FILE} > ${OUTPUT_FILE}
echo "The file ${INPUT_FILE} has been processed and saved as ${OUTPUT_FILE}"

# drop the third column ('Instance') from the csv file (it does not exist in the other files)
# Use awk to drop the third column
awk -F, 'BEGIN {OFS=","} { $3=""; sub(",,", ","); print }' ${OUTPUT_FILE} > ${INPUT_FILE}

rm ${OUTPUT_FILE}

echo "The file ${INPUT_FILE} has been processed, the third column has been removed:"
head -3 $INPUT_FILE

UID,DBType,Instance,DBName,Schema,Table,Column,ColumnType
30230,MSSQL,R0015702\RCHSCOMP02,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InRedStateMilliseconds,int
30230,MSSQL,R0015702\RCHSCOMP02,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InYellowStateMilliseconds,int
The file data/mssql_allprod_collumns.csv has been processed and saved as data/mssql_allprod_collumns_no_third_column.csv.tmp
The file data/mssql_allprod_collumns.csv has been processed, the third column has been removed:
﻿UID,DBType,DBName,Schema,Table,Column,ColumnType
30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InRedStateMilliseconds,int
30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InYellowStateMilliseconds,int


In [55]:
%%bash
# prepare Oracle data
INPUT_FILE="data/oracle_metadata.csv"
OUTPUT_FILE="data/oracle_metadata.csv.tmp"

# the data of one table contains strange column names, remove its columns
sed '/RFA772203/d' $INPUT_FILE > $OUTPUT_FILE

mv $OUTPUT_FILE $INPUT_FILE
rm -f $OUTPUT_FILE

In [56]:
%%bash
# create the dataset to use for the analysis

# the number of records to use for each DB type, set to '-1' to use all records
NR_OF_RECORDS_EACH=5000

TARGET_FILE="data/db_metadata.csv"
echo "Creating the dataset..."

# drop the first line (column names) from the files (except the one from MSSQL)
tail -n +2 data/oracle_metadata.csv > data/oracle_metadata.csv.tmp
tail -n +2 data/mysql_metadata.csv > data/mysql_metadata.csv.tmp

mv data/oracle_metadata.csv.tmp data/oracle_metadata.csv
mv data/mysql_metadata.csv.tmp data/mysql_metadata.csv

if [ $NR_OF_RECORDS_EACH -gt 0 ]; then
    echo "Creating a small dataset with ${NR_OF_RECORDS_EACH} records of each files"
    head -${NR_OF_RECORDS_EACH} data/mssql_allprod_collumns.csv  >  ${TARGET_FILE}
    head -${NR_OF_RECORDS_EACH} data/oracle_metadata.csv         >> ${TARGET_FILE}
    head -${NR_OF_RECORDS_EACH} data/mysql_metadata.csv          >> ${TARGET_FILE}
else
    echo "Creating a dataset with all records of each files"
    cat data/mssql_allprod_collumns.csv data/oracle_metadata.csv data/mysql_metadata.csv > ${TARGET_FILE}
fi

NR_OF_LINES=$(wc -l ${TARGET_FILE})
echo "new dataset contains ${NR_OF_LINES}"

Creating the dataset...


Creating a small dataset with 5000 records of each files
new dataset contains    15000 data/db_metadata.csv


In [57]:
# Importing the required libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import LatentDirichletAllocation
import wordninja
# from nltk.corpus import stopwords

In [58]:
# Load the data
df = pd.read_csv('data/db_metadata.csv')
display(df.head())

# for more flexibility when selecting the data file, we rename the columns to be the same for all
new_column_names = {
    df.columns[0]: 'UID',  
    df.columns[1]: 'DB_TYPE',
    df.columns[2]: 'DB_NAME',
    df.columns[3]: 'SCHEMA',
    df.columns[4]: 'TABLE_NAME',
    df.columns[5]: 'COLUMN_NAME',
    df.columns[6]: 'DATA_TYPE',
}
df.rename(columns=new_column_names, inplace=True)

display(df.head())

,UID,DBType,DBName,Schema,Table,Column,ColumnType
0,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InRedStateMilliseconds,int
1,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InYellowStateMilliseconds,int
2,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InDisabledStateMilliseconds,int
3,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InPlannedMaintenanceMilliseconds,int
4,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InUnplannedMaintenanceMilliseconds,int


,UID,DB_TYPE,DB_NAME,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE
0,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InRedStateMilliseconds,int
1,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InYellowStateMilliseconds,int
2,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InDisabledStateMilliseconds,int
3,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InPlannedMaintenanceMilliseconds,int
4,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InUnplannedMaintenanceMilliseconds,int


# Utilities

In [69]:
from lingua import Language, LanguageDetectorBuilder

# build a language detector to detect English and German
langdetector = LanguageDetectorBuilder.from_languages(
        Language.ENGLISH, 
        Language.GERMAN
    ).build()

# function to detect if a word is German
def isGerman(word):
    detected_lang = langdetector.detect_language_of(word)
    if detected_lang == Language.GERMAN:
        return True
    else:
        return False

### Prepare the data

In [97]:
# drop columns of known special tables (e.g. technical tables used by flyway, liquibase, quartz, jobrunr)
df = df[~df['TABLE_NAME'].str.lower().str.contains('flyway') 
        & ~df['TABLE_NAME'].str.lower().str.contains('qrtz')
        & ~df['TABLE_NAME'].str.lower().str.contains('shedlock')
        & ~df['TABLE_NAME'].str.lower().str.contains('jobrunr')]
df = df.drop(df[(df['TABLE_NAME'].str.lower() == 'databasechangelog') | (df['TABLE_NAME'].str.lower() == 'databasechangeloglock')].index)


In [98]:
display(df.head())

,UID,DB_TYPE,DB_NAME,SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,DOCUMENT_CONTENT
0,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InRedStateMilliseconds,int,red state milliseconds
1,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InYellowStateMilliseconds,int,yellow state milliseconds
2,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InDisabledStateMilliseconds,int,disabled state milliseconds
3,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InPlannedMaintenanceMilliseconds,int,planned maintenance milliseconds
4,30230,MSSQL,OperationsManagerDW,State,StateHourly_D461F6AFA87B4259B908B29DA01EDE5C,InUnplannedMaintenanceMilliseconds,int,unplanned maintenance milliseconds


In [102]:

# function to split a string into words and filter out short words (less than 4 characters) and numbers
def splitToWordsAndFilter(x):
    if pd.notnull(x) and x:
        if isGerman(x):
            # currrntly we don't have a German word-splitter, so we at least use the underscore to split the words
            words = x.split('_')
        else:
            words = wordninja.split(x)
        return ' '.join([word.lower() for word in words if len(word) > 3 and not word.isdigit()])
    return ''

# split the column names 
df['DOCUMENT_CONTENT'] = df['COLUMN_NAME'].apply(
    lambda x: splitToWordsAndFilter(x)
)

# - Concatenate 'COLUMN_NAME' values for rows with the same 'DB_NAME'/'TABLE_NAME' combination
# - lowercase the text
df_grouped = df.groupby(['UID', 'DB_TYPE', 'DB_NAME', 'TABLE_NAME'])['DOCUMENT_CONTENT'].agg(
    lambda x: ' '.join(x.str.lower())
).reset_index()


df_grouped['TABLE_NAME_CONTENT'] = df_grouped['TABLE_NAME'].apply(
    # don't fail if resulting string is empty
    lambda x: splitToWordsAndFilter(x)
)

df_grouped['DOCUMENT_CONTENT'] = df_grouped['TABLE_NAME_CONTENT'] + ' ' + df_grouped['DOCUMENT_CONTENT']


In [103]:
display(df_grouped.head())

,UID,DB_TYPE,DB_NAME,TABLE_NAME,DOCUMENT_CONTENT,TABLE_NAME_CONTENT
0,100,Oracle,PCRMA01,ADVISORYOUTPUT_2,advisory output financial analysis modified m...,advisory output
1,100,Oracle,PCRMA01,BAK_PRICING_2,pricing deleted frozen modified modifieddate...,pricing
2,100,Oracle,PCRMA01,CMZKE_KUNDE,cmzke kunde changed changed created created do...,cmzke kunde
3,100,Oracle,PCRMA01,CREDITWORTHINESSINDICATORS_2,credit worthiness indicators modified modifie...,credit worthiness indicators
4,100,Oracle,PCRMA01,DEBTCAPACITY_IFBC_2,debt capacity financial analysis modified mod...,debt capacity


In [104]:
# write a CSV with the table names and their split names
df_grouped[['TABLE_NAME', 'TABLE_NAME_CONTENT']].to_csv('table_name_content.csv', index=False)

### Calculate TF-IDF

In [105]:
from spacy.lang.de.stop_words import STOP_WORDS as de_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

# some custom stop words
custom_stop_words = ['item', 'name', 'type', 'code', 'id', 'key', 'value', 'date', 'time', 'status', 'flag', 'description', 'comment', 'number', 'size', 'length', 'version', 'order', 'count', 'level', 'class', 'group', 'status', 'state', 'row', 'str', 'vid', 'ext', 'set', 'rid']

# we have english and german words in the data set, so combine the stop words of both languages
final_stopwords_list = list(de_stop) + list(en_stop) + custom_stop_words

print(len(final_stopwords_list))

899


TODO:
- ~~also remove all numbers from the text~~
- ~~lowercase the text~~
- ~~add table name to DOCUMENT_CONTENT~~
- ~~german vs english~~
~~'CMZKE_KUNDE' -> KUN, should be: KUNDE~~
- german to english translation
- lemmatizing ([germalemma](https://github.com/WZBSocialScienceCenter/germalemma))

In [106]:

# Create a document-term matrix using TF-IDF
vectorizer = TfidfVectorizer(stop_words=final_stopwords_list, min_df=4, max_df=0.75, max_features=5000)
doc_term_matrix = vectorizer.fit_transform(df_grouped['DOCUMENT_CONTENT'].values)

print(doc_term_matrix.shape)

(1401, 763)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(


In [107]:
num_clusters = 5

lda = LatentDirichletAllocation(n_components=num_clusters, learning_method='online', random_state=42)
doc_cluster_matrix = lda.fit_transform(doc_term_matrix)

col_names = ['Cluster' + str(i) for i in range(num_clusters)]

doc_cluster_df = pd.DataFrame(doc_cluster_matrix, columns=col_names)
display(doc_cluster_df.head()) 


,Cluster0,Cluster1,Cluster2,Cluster3,Cluster4
0,0.055642,0.056213,0.776780,0.055633,0.055732
1,0.059022,0.059827,0.762994,0.058986,0.059172
2,0.069716,0.071164,0.699396,0.089533,0.070191
3,0.059200,0.059796,0.763576,0.058651,0.058777
4,0.053063,0.053767,0.787261,0.052914,0.052996


In [108]:
#Visualize and analyze reuslts
import pyLDAvis
import pyLDAvis.lda_model

prepared_data = pyLDAvis.lda_model.prepare(lda, doc_term_matrix, vectorizer, mds='tsne', sort_topics=False, n_jobs = -1)
word_info = prepared_data.topic_info

#Print top 30 keywords
for topic in word_info.loc[word_info.Category != 'Default'].Category.unique():
    print(topic)
    print(word_info.loc[word_info.Category.isin([topic])].sort_values('logprob', ascending = False).Term.values[:30])
    print()

#To save prepared_data in an html panel (using https://panel.holoviz.org/)
pyLDAvis.save_html(prepared_data, 'panel.html')
print("Panel saved to panel.html")

/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=78949) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=78949) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=78949) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=78949) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarnin

Topic1
['element' 'milliseconds' 'product' 'pruefperson' 'quantity' 'message'
 'rate' 'price' 'account' 'alias' 'bank' 'data' 'scope' 'access'
 'position' 'request' 'execution' 'pwc01' 'device' 'contract'
 'bezeichnung' 'property' 'reference' 'currency' 'interval' 'arrangement'
 'maintenance' 'configuration' 'health' 'valid']

Topic2
['created' 'user' 'object' 'modified' 'host' 'lock' 'file' 'attribute'
 'entity' 'data' 'resource' 'fact' 'content' 'audit' 'elementvalue'
 'elementname' 'text' 'rule' 'active' 'timestamp' 'definition'
 'lastmodifieddate' 'intflags' 'work' 'token' 'session' 'entry' 'document'
 'username' 'lifecycle']

Topic3
['updated' 'data' 'partner' 'partition' 'datum' 'dtdate' 'service'
 'source' 'valid' 'rest' 'modifieddate' 'created' 'dates' 'aggregation'
 'modified' 'virtual' 'relationship' 'deletion' 'touched' 'xdateinserted'
 'inserted' 'marked' 'xmldata' 'active' 'book' 'server' 'calculated'
 'corder' 'nachfolger' 'pruefperson']

Topic4
['event' 'lifecycle' 'asse